# Asensio & Ferragut 2002 experiment

This notebook implements experiment of [1] using ```ngen-kutral``` framework.

In [1]:
import numpy as np
import wildfire
from wildfire.utils.functions import G
from wildfire.utils import plots
import ipywidgets as widgets

## Physical parameters



In [2]:
# Physical arameters for the model
physical_parameters = {    
    'kap': 1e-1, # diffusion coefficient
    'eps': 3e-1, # inverse of activation energy
    'upc': 3, # u phase change
    'q': 1, # reaction heat
    'alp': 1e-3, # natural convection,
    # Domain
    'x_lim': (0, 90), # x-axis domain 
    'y_lim': (0, 90), # y-axis domain
    't_lim': (0, 30) # time domain
}

## Create Wildfire 

In [3]:
wildfire_ = wildfire.Fire(**physical_parameters)

## Numerical Parameters

* Temperature initial condition: $u_0(x,y)=6\exp(-((x-20)^2 + (y-20)^2) / 20)$
* Fuel initial condition: $\beta_0(x,y) \sim \text{Uniform}(0,1)$
* Wind + topography: $\mathbf{v}(x, y, t) = (\cos(\pi / 4), \sin(\pi /4))$

In [4]:
# Temperature initial condition
u0 = lambda x, y: 6 * G(x-20, y-20, 20)

# Fuel initial condition
def b0(x, y):
  np.random.seed(666)
  br = lambda x, y: np.round(np.random.uniform(size=(x.shape)), decimals=2)
  B = br(x, y)
  x_rows, x_cols = x.shape
  y_rows, y_cols = y.shape
  B[0,:] = np.zeros(x_cols)
  B[-1,:] = np.zeros(x_cols)
  B[:,0] = np.zeros(y_rows)
  B[:,-1] = np.zeros(y_rows)
  return B

# Wind effect
gamma = 1
w1 = lambda x, y, t: gamma * np.cos(np.pi/4 + x * 0)
w2 = lambda x, y, t: gamma * np.sin(np.pi/4 + x * 0)

V = (w1, w2)

### Widgets

In [5]:
# Widget to select space method
space_method_dp = widgets.Dropdown(
    options=[('Finite Difference', 'fd'), ('Fast Fourier Transform', 'fft')], 
    value='fd', 
    description="Space Method"
)

# Widget to define number of x nodes
space_nx_text = widgets.BoundedIntText(
    value=128,
    min=64,
    max=256,
    step=1,
    description=r"$N_x$:",
    disabled=False
)

# Widget to define number of y nodes
space_ny_text = widgets.BoundedIntText(
    value=128,
    min=64,
    max=256,
    step=1,
    description=r"$N_y$:",
    disabled=False
)

# Widget to select time method
time_method_dp = widgets.Dropdown(
    options=[('Euler Method', 'Euler'), ('Runge-Kutta of 4th order', 'RK4')], 
    value='RK4', 
    description="Time Method"
)

# Widget to define number of t nodes
time_nt_text = widgets.BoundedIntText(
    value=100,
    min=100,
    max=1000,
    step=1,
    description=r"$N_t$:",
    disabled=False
)

# Checkbox to select last approximation
time_last_cb = widgets.Checkbox(
        value=False,
        description='Last approximation:',
        disabled=False,
        indent=False
)
out = widgets.Output()

In [6]:
def showSimulation(t, X, Y, U, B, V, k=0):
    plots.UBs(k, t, X, Y, U, B, V)

In [7]:
def simulation(space_method, Nx, Ny, time_method, Nt):
    t, X, Y, U, B = wildfire_.solvePDE(Nx, Ny, Nt, u0, b0, V, space_method, time_method, last=False)
    time_steps_sl = widgets.IntSlider(value=0, min=0, max=Nt, step=1, continuous_update=False)
    widgets.interact(showSimulation, t=widgets.fixed(t), X=widgets.fixed(X), Y=widgets.fixed(Y),
        U=widgets.fixed(U), B=widgets.fixed(B), V=widgets.fixed(V), k=time_steps_sl)

In [9]:
widget_ = widgets.interact_manual(simulation, space_method=space_method_dp,
    Nx=space_nx_text, Ny=space_ny_text, time_method=time_method_dp, Nt=time_nt_text)

interactive(children=(Dropdown(description='Space Method', options=(('Finite Difference', 'fd'), ('Fast Fourie…

In [10]:
#widget_

In [13]:
# TODO: Make a button
"""
space_box = widgets.VBox(widget_.children[:3])
time_box  = widgets.VBox(widget_.children[3:5])

# defining a list with the contents of our windows
children = [space_box, time_box]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'Space parameters')
tab.set_title(1, 'Time parameters')
ui = widgets.VBox([tab])

accordian = widgets.Accordion(children=[ui])
accordian.set_title(0, 'Numerical Parameters')
out = widget_.children[-1]
UI = widgets.VBox([accordian, out])
display(UI)
"""

"\nspace_box = widgets.VBox(widget_.children[:3])\ntime_box  = widgets.VBox(widget_.children[3:5])\n\n# defining a list with the contents of our windows\nchildren = [space_box, time_box]\n# initializing a tab\ntab = widgets.Tab()\n# setting the tab windows \ntab.children = children\n# changing the title of the first and second window\ntab.set_title(0, 'Space parameters')\ntab.set_title(1, 'Time parameters')\nui = widgets.VBox([tab])\n\naccordian = widgets.Accordion(children=[ui])\naccordian.set_title(0, 'Numerical Parameters')\nout = widget_.children[-1]\nUI = widgets.VBox([accordian, out])\ndisplay(UI)\n"

# References

* [1] Asensio, M. I., & Ferragut, L. (2002). On a wildland fire model with radiation. International Journal for Numerical Methods in Engineering, 54(1), 137-157. https://doi.org/10.1002/nme.420